## Neural Collaborative Filtering
Neural Collaborative Filtering (NCF) is primarily used for Collaborative Filtering, which suggests items based on similarities in user preferences, without relying on extra features.

NCF is considered to be an advanced version of Matrix Factorization which not only captures linear relationships, but also non-linear relationships.NCF uses Matrix Factorization in combination with a Neural Network for collaborative filtering.

### Architecture
![image](./images/ncf.webp)

Neural Collaborative Filtering (NCF) utilizes a neural network with two branches that eventually converge to produce a single output. The NCF architecture consists of several key components:

### 1. Input and Output

NCF processes one User-Item pair at a time. The inputs are:

- **One-Hot Encoded (OHE) representations** of the User and Item, provided separately.

The output can be:

- **Expected rating** the user will give to the item, if explicit feedback is used during training.
- **Probability** that the user will interact with the item, if implicit feedback is used.

### 2. Embeddings Layer

This layer generates meaningful embeddings for both users and items. There are multiple embedding layers tailored for different branches of the NCF model:

- **No advanced techniques** are applied in this layer.

### 3. Architecture Breakdown

The NCF architecture is divided into two main sections:

- **Generalized Matrix Factorization (GMF)**:
  - This component performs a straightforward multiplication of the user and item embedding matrices derived from the embeddings layer.
  - It captures **linear relationships** and is akin to traditional Matrix Factorization.

- **Multi-Layer Perceptron (MLP)**:
  - This is a standard Deep Neural Network (DNN) that processes the concatenated user and item embeddings through multiple hidden layers with ReLU activation functions.
  - It captures **non-linear patterns** in the data.

### 4. Combining Outputs

The outputs from GMF and MLP are concatenated, and a hidden layer is applied to merge these outputs. This combined layer is known as **NeuMF**. A sigmoid activation function follows to produce the final output.

### 5. Summary

Thus, NCF integrates:

- **Embeddings Layer**
- **GMF**
- **MLP**
- **NeuMF**

**Note:** The user and item embeddings for GMF and MLP are distinct. Therefore, the embeddings used in GMF differ from those used in MLP, resulting in a total of four separate embeddings in the entire architecture. The loss function employed is **Logloss**, similar to Logistic Regression, which is designed for implicit feedback.

## Implementation

#### 1. Imports

In [1]:
!pip install LibRecommender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd


from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF
from libreco.evaluation import evaluate

Instructions for updating:
non-resource variables are not supported in the long term


### 2. Dataset
Movielens data

In [3]:
import random
import tensorflow as tf
from pathlib import Path

def load_ml_1m():
    # download and extract zip file
    tf.keras.utils.get_file(
        "ml-1m.zip",
        "http://files.grouplens.org/datasets/movielens/ml-1m.zip",
        cache_dir=".",
        cache_subdir=".",
        extract=True,
    )
    # read and merge data into same table
    cur_path = Path(".").absolute()
    ratings = pd.read_csv(
        cur_path / "ml-1m" / "ratings.dat",
        sep="::",
        usecols=[0, 1, 2, 3],
        names=["user", "item", "rating", "time"],
    )
    users = pd.read_csv(
        cur_path / "ml-1m" / "users.dat",
        sep="::",
        usecols=[0, 1, 2, 3],
        names=["user", "sex", "age", "occupation"],
    )
    items = pd.read_csv(
        cur_path / "ml-1m" / "movies.dat",
        sep="::",
        usecols=[0, 2],
        names=["item", "genre"],
        encoding="iso-8859-1",
    )
    items[["genre1", "genre2", "genre3"]] = (
        items["genre"].str.split(r"|", expand=True).fillna("missing").iloc[:, :3]
    )
    items.drop("genre", axis=1, inplace=True)
    data = ratings.merge(users, on="user").merge(items, on="item")
    data.rename(columns={"rating": "label"}, inplace=True)
    # random shuffle data
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    return data

In [4]:
data = load_ml_1m()

data.shape

5917549/5917549 [==============================] - 0s 0us/step


<ipython-input-3-29da8184df07>:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(
<ipython-input-3-29da8184df07>:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv(
<ipython-input-3-29da8184df07>:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  items = pd.read_csv(


(1000209, 10)

In [5]:
data.head()

,user,item,label,time,sex,age,occupation,genre1,genre2,genre3
0,5755,184,3,958280246,F,35,2,Drama,missing,missing
1,4585,519,3,964321944,M,35,7,Sci-Fi,Thriller,missing
2,1503,3114,4,974762175,M,25,12,Animation,Children's,Comedy
3,2166,648,4,974614593,M,1,10,Action,Adventure,Mystery
4,3201,2178,5,968626301,M,45,7,Thriller,missing,missing


In [6]:
data.columns

Index(['user', 'item', 'label', 'time', 'sex', 'age', 'occupation', 'genre1',
       'genre2', 'genre3'],
      dtype='object')

In [7]:
data = data[['user', 'item', 'label', 'time']]

In [8]:
data.head()

,user,item,label,time
0,5755,184,3,958280246
1,4585,519,3,964321944
2,1503,3114,4,974762175
3,2166,648,4,974614593
4,3201,2178,5,968626301


In [9]:
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1])

In [10]:
# Convert the pandas dataframe into a compatible datatype for LibRecommender
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

- DatasetPure: This is indicative of the fact we are not using any other feature other than the interaction between the user and an item. Hence, this function builds the datasets from a Pure Collaborative Filtering perspective.

## 3. NCF

Look: https://librecommender.readthedocs.io/en/latest/user_guide/model_train.html

In [11]:
ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=64,
    n_epochs=50,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)

### Hyper-parameters
- Task = `rating` and `ranking`

- `Rating` is usually used when we have a dataset around explicit feedback (direct rating, starts given by customers)

- The `ranking` is used when the dataset has implicit feedback (customer clicks, opens a webpage, etc)

- `data_info` holds meta-information about the training dataset

- `embedding_size`= Embedding size for user and item embeddings


In [12]:
# monitor metrics on eval data during training
ncf.fit(
    train_data,
    neg_sampling=False, #for rating, this param is false else True
    verbose=2,
    eval_data=eval_data,
    metrics=["loss"],
)

/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
Instructions for updating:
Colocations handled automatically by placer.


Training start time: 2024-07-22 19:08:20


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
train: 100%|██████████| 782/782 [00:06<00:00, 126.01it/s]


Epoch 1 elapsed: 6.212s
	 train_loss: 1.8648


eval_pointwise: 100%|██████████| 13/13 [00:00<00:00, 172.56it/s]


...

	 eval rmse: 1.0466


train: 100%|██████████| 782/782 [00:04<00:00, 187.18it/s]


Epoch 50 elapsed: 4.187s
	 train_loss: 0.259


eval_pointwise: 100%|██████████| 13/13 [00:00<00:00, 402.06it/s]

	 eval rmse: 1.0455


### 4. Evaluation

In [13]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],
)
#for implicit feedback, metrics like precision@k, recall@k, ndcg can be used

eval_pointwise: 100%|██████████| 13/13 [00:00<00:00, 346.22it/s]


{'loss': 1.0433564}

### 5. Prediction Time

In [14]:
# predict preference of user 5755 to item 110
ncf.predict(user=5755, item=110)

array([3.6730342], dtype=float32)

In [15]:
# recommend 10-items for user 5755
ncf.recommend_user(user=5755, n_rec=10)

{5755: array([3746, 1235,  599, 3718, 1922, 2858,  608,  922, 3338, 1203])}